In [1]:
# -*- coding:utf-8 -*-

import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm


def count_script(payload):
    return str(payload).lower().count('script')


def count_java(payload):
    return str(payload).lower().count('java')


def count_angle_brackets(payload):
    return str(payload).lower().count('<')


def count_parentheses(payload):
    return str(payload).lower().count('(')



def count_alert(payload):
    return str(payload).lower().count('alert')


def count_percent(payload):
    return str(payload).lower().count('%')


def judge_cases_exception(payload):
    payload_lowercase = str(payload).lower()
    if payload_lowercase == str(payload):
        return 0
    return 1

def have_sensitive_words(payload):
    words_list = ['fuck', 'xss', 'shit']
    for word in words_list:
        if word in str(payload).lower():
            return 1 
    return 0



In [2]:
XSS_label = []

normal_data = pd.read_csv('dmzo_nomal.csv', header=None, names=['payload'])
for i in range(len(normal_data)):
    XSS_label.append(0)
normal_data['XSS'] = XSS_label

XSS_label = []
xssed_data = pd.read_csv('xssed.csv', header=None, names=['payload'])
for i in range(len(xssed_data)):
    XSS_label.append('1')
xssed_data['XSS'] = XSS_label

data = pd.concat([normal_data, xssed_data], keys=['payload', 'XSS'])
data = shuffle(data)
# print(data)

features = []
labels = []
for i in range(len(data)):
    payload = data['payload'][i]
    features.append([
                     count_alert(payload),
                     count_script(payload),
                     count_java(payload),
                     count_angle_brackets(payload),
                     count_parentheses(payload),
                     count_percent(payload),
                     judge_cases_exception(payload),
                     have_sensitive_words(payload)
                    ])
    labels.append(data['XSS'][i])

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3,
                                                    random_state=0, stratify=labels)

svc = svm.SVC(kernel='linear', C=1.0, gamma='auto').fit(X_train, y_train)  # 线性核
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=1.0).fit(X_train, y_train)  # 径向基核
poly_svc = svm.SVC(kernel='poly', degree=3, C=1.0, gamma='auto').fit(X_train, y_train)  # 多项式核
print('Finished training.\n');

Finished training.



In [3]:
print(svc.score(X_test, y_test))
print(rbf_svc.score(X_test, y_test))
print(poly_svc.score(X_test, y_test))

0.9916195372750642
0.9913110539845759
0.9841645244215939
